![Reflection Agent Pattern](react.png)

### A ReAct agent works in cycles:

Step	Description
1️⃣	Receive input (user query or task)

2️⃣	Reason: The model thinks aloud — plans what to do next

3️⃣	Act: Executes an action (e.g., tool call, web search, Python code)

4️⃣	Observe: Reads the result from the tool

5️⃣	Repeat: Uses new information to reason again

6️⃣	Finish: Returns a final answer

### React agent in lancghain vs langraph
### 1. Big Picture: LangChain vs LangGraph
Framework	What it is	Core Purpose
LangChain	A framework for building LLM applications using chains, tools, memory, and agents.	Makes it easy to connect LLMs with APIs, prompts, vector stores, etc.
LangGraph	A layer on top of LangChain that adds graph-based state management and multi-step agent control.	Enables building stateful, reactive, multi-agent workflows that can reason, act, and maintain memory across multiple nodes.

✅ In short:

LangChain → simple, linear workflows.

LangGraph → advanced, dynamic, stateful agent systems (with loops, branching, and persistence).

### 🧩 2. What’s a ReAct Agent in Each Framework?
🔹 In LangChain

ReAct Agent = an AgentExecutor that runs a loop internally, hidden inside Python logic.

You can’t easily visualize or modify its inner loop — it’s managed for you.

🔹 In LangGraph

ReAct Agent = a StateGraph where each step of the loop (Reason → Act → Observe) is a node you control, modify, visualize, or persist.

You can even connect multiple agents together, share state, or branch logic dynamically.

### ⚙️ 3. Why LangGraph’s ReAct Is Better than LangChain’s
Feature	LangChain	LangGraph	Why It Matters
Control Flow	Mostly linear (Chains → sequential calls).	Graph-based (nodes, edges, conditions, loops).	You can create branching and cyclic workflows (like ReAct loops).
State Management	Implicit; state lost between steps unless using memory objects.	Explicit, shared State that automatically merges and persists.	Keeps track of everything (tools used, thoughts, history).
Persistence	Manual; you must save states or use Memory.	Built-in checkpointing (Redis, file, or DB).	You can pause/resume multi-step workflows.
Multi-agent Support	Hard to manage multiple agents talking to each other.	Natural — each agent can be a node in the graph.	Enables collaborative agents (Planner, Worker, Evaluator).
Visualization	None (just console traces).	Graph visualization via .get_graph().draw_png().	Easier debugging and documentation.
Error Handling	Needs try/except logic.	Built-in retry/fallback nodes.	Stable workflows.
Reactivity	Limited (static chains).	Dynamic (graph reacts to state changes).	True ReAct loop control.

### React agent in langchain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain.agents import initialize_agent, tool
from langchain_community.tools import TavilySearchResults
import datetime

load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

search_tool = TavilySearchResults(search_depth="basic")

@tool
def get_system_time(format: str = "%Y-%m-%d %H:%M:%S"):
    """ Returns the current date and time in the specified format """

    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime(format)
    return formatted_time


tools = [search_tool, get_system_time]

agent = initialize_agent(tools=tools, llm=llm, agent="zero-shot-react-description", verbose=True)

agent.invoke("When was SpaceX's last launch and how many days ago was that from this instant")


ImportError: cannot import name 'AgentType' from 'langchain.agents' (d:\langraph\.venv\Lib\site-packages\langchain\agents\__init__.py)

### 🧩 What Happens Internally

LangChain’s built-in ReAct agent handles the loop internally.

You can’t control when it reasons, acts, or observes — it’s a black box.

You can’t inspect state transitions or resume halfway through.

### How to use a React Agent in langrpah

In [3]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain import hub

# Load a ReAct prompt template
prompt = hub.pull("hwchase17/react")

# Define a tool
def search(query: str):
    return "The population of Japan is 125.7 million."

tools = [Tool(name="Search", func=search, description="Useful for looking up information.")]

# Create agent using ReAct template
llm = ChatOpenAI(model="gpt-4o-mini")
agent = create_react_agent(llm, tools, prompt)

# Create executor (this runs the reasoning loop)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Run the agent
agent_executor.invoke({"input": "What is Japan’s population plus 2?"})


ImportError: cannot import name 'create_react_agent' from 'langchain.agents' (d:\langraph\.venv\Lib\site-packages\langchain\agents\__init__.py)